## 트랜스포머
- RNN의 속성을 따라감 + 트랜스포머
- 어텐션 메커니즘
    - 멀티 헤드 어텐션으로 문맥 임베딩 인코딩하기
    - 디코더와 마스크드 멀티 헤드 어텐션
    - 위치 인코딩 및 층 정규화

- K, Q, V
    - Q * K (내적, dot product) => 두 벡터의 방향 유사도
        - 병렬 처리에 "약간" 더 좋음
        - Q(질문), K(키)가 각각의 벡터에서 얼마나 가까운가?
        - "이 Q(쿼리)가 각 K(토큰)에 얼마나 집중(주목)해야 하나요?"

    - softmax(내적 값): 가중치 정규화
    - 가중치 × V → 최종 표현



In [ ]:
%pip install transformers datasets accelerate evaluate

## Chat형도

> 일반 텍스트 -> 기형도 시인의 문체를 약간이나마 따라하는 출력 결과를 보고 싶음

### 설계
```Python
1. 기존에 학습된 모델이 필요함(Base 모델)
2. NLP의 경우 tokenizer를 결정, 입력 데이터에 따라 숫자로 변환하는 알고리즘
3. 임베딩을 진행, 숫자로 변환
4. 데이터를 Base 모델에 학습(파인 튜닝)
5. 평가를 진행(샘플 텍스트를 몇 개 작성)
6. 확인(그래프, 숫자, 표)
7. 반복

In [ ]:
class KoBARTFineTuningConfig:
    # 모델 설정
    MODEL_NAME = "gogamza/kobart-base-v2"  # 한국어 BART 모델

    # 훈련 하이퍼파라미터
    NUM_EPOCHS = 3
    BATCH_SIZE = 8
    LEARNING_RATE = 2e-5
    WARMUP_STEPS = 500
    MAX_SOURCE_LENGTH = 512  # 입력 텍스트 최대 길이
    MAX_TARGET_LENGTH = 128  # 출력 텍스트 최대 길이

    # 출력 설정
    OUTPUT_DIR = "./kobart-finetuned"
    LOGGING_DIR = "./logs"

    # 평가 설정
    EVAL_STEPS = 500
    SAVE_STEPS = 500
    LOGGING_STEPS = 100

    # 생성 설정 (추론 시)
    NUM_BEAMS = 4
    MAX_GENERATION_LENGTH = 128
    EARLY_STOPPING = True

In [ ]:
import numpy as np
import json
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    BartForConditionalGeneration,
    PreTrainedTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)

In [ ]:
# ============================================================================
# 2. 데이터셋 준비 및 전처리
# ============================================================================
def load_custom_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    dataset_dict = DatasetDict(
        {
            split: Dataset.from_list(data[split])
            for split in ["train", "validation", "test"]
        }
    )

    return dataset_dict

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 23
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 4
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 6
    })
})

In [ ]:
# ============================================================================
# 3. 토크나이저 및 모델 로드
# ============================================================================
def load_tokenizer_and_model(model_name=None):
    if model_name is None:
        model_name = KoBARTFineTuningConfig.MODEL_NAME

    print(f"모델 로딩 중: {model_name}")

    # 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(model_name)

    print(f"토크나이저 어휘 크기: {len(tokenizer)}")
    print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

    return tokenizer, model

In [ ]:
# ============================================================================
# 4. 데이터 전처리 함수
# ============================================================================
def preprocess_function(
    examples,
    tokenizer,
    max_source_length,
    max_target_length,
):

    # 입력 텍스트 (document) 토큰화
    model_inputs = tokenizer(
        examples["document"],
        max_length=max_source_length,
        truncation=True,
        padding="max_length",
        return_token_type_ids=False,  # BART는 token_type_ids를 사용하지 않음
    )

    # 타겟 텍스트 (summary) 토큰화
    # 최신 transformers에서는 text_target 인자 사용 (as_target_tokenizer 대신)
    labels = tokenizer(
        text_target=examples["summary"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
    )

    # labels를 model_inputs에 추가
    model_inputs["labels"] = labels["input_ids"]

    # token_type_ids가 있다면 제거 (BART는 사용하지 않음)
    if "token_type_ids" in model_inputs:
        del model_inputs["token_type_ids"]

    return model_inputs

In [ ]:
def prepare_dataset(dataset, tokenizer):
    config = KoBARTFineTuningConfig

    # 각 split에 대해 전처리 적용
    tokenized_dataset = dataset.map(
        lambda examples: preprocess_function(
            examples, tokenizer, config.MAX_SOURCE_LENGTH, config.MAX_TARGET_LENGTH
        ),
        batched=True,
        remove_columns=dataset["train"].column_names,  # 원본 컬럼 제거
    )

    return tokenized_dataset

### 학습
```Python
1. 데이터셋 준비
2. 토크나이저 및 모델
3. 데이터 전처리
4. 데이터 폴레이터
5. 데이터 파라미터 결정
6. 훈련
7. 평가
8. 저장

In [13]:
import numpy as np
import json
import torch
import evaluate
import nltk
nltk.download("punkt")
from datasets import Dataset, DatasetDict
from transformers import (
    BartForConditionalGeneration,
    PreTrainedTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)


# ============================================================================
# 1. 설정 및 초기화
# ============================================================================


class KoBARTFineTuningConfig:
    # 모델 설정
    MODEL_NAME = "gogamza/kobart-base-v2"  # 한국어 BART 모델

    # 훈련 하이퍼파라미터
    NUM_EPOCHS = 3
    BATCH_SIZE = 8
    LEARNING_RATE = 2e-5
    WARMUP_STEPS = 500
    MAX_SOURCE_LENGTH = 512  # 입력 텍스트 최대 길이
    MAX_TARGET_LENGTH = 128  # 출력 텍스트 최대 길이

    # 출력 설정
    OUTPUT_DIR = "./kobart-finetuned"
    LOGGING_DIR = "./logs"

    # 평가 설정
    EVAL_STEPS = 500
    SAVE_STEPS = 500
    LOGGING_STEPS = 100

    # 생성 설정 (추론 시)
    NUM_BEAMS = 4
    MAX_GENERATION_LENGTH = 128
    EARLY_STOPPING = True


# ============================================================================
# 2. 데이터셋 준비 및 전처리
# ============================================================================
def load_custom_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    dataset_dict = DatasetDict(
        {
            split: Dataset.from_list(data[split])
            for split in ["train", "validation", "test"]
        }
    )

    return dataset_dict


# ============================================================================
# 3. 토크나이저 및 모델 로드
# ============================================================================
def load_tokenizer_and_model(model_name=None):
    if model_name is None:
        model_name = KoBARTFineTuningConfig.MODEL_NAME

    print(f"모델 로딩 중: {model_name}")

    # 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(model_name)

    print(f"토크나이저 어휘 크기: {len(tokenizer)}")
    print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

    return tokenizer, model


# ============================================================================
# 4. 데이터 전처리 함수
# ============================================================================
def preprocess_function(
    examples,
    tokenizer,
    max_source_length,
    max_target_length,
):

    # 입력 텍스트 (document) 토큰화
    model_inputs = tokenizer(
        examples["document"],
        max_length=max_source_length,
        truncation=True,
        padding="max_length",
        return_token_type_ids=False,  # BART는 token_type_ids를 사용하지 않음
    )

    # 타겟 텍스트 (summary) 토큰화
    # 최신 transformers에서는 text_target 인자 사용 (as_target_tokenizer 대신)
    labels = tokenizer(
        text_target=examples["summary"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
    )

    # labels를 model_inputs에 추가
    model_inputs["labels"] = labels["input_ids"]

    # token_type_ids가 있다면 제거 (BART는 사용하지 않음)
    if "token_type_ids" in model_inputs:
        del model_inputs["token_type_ids"]

    return model_inputs


def prepare_dataset(dataset, tokenizer):
    config = KoBARTFineTuningConfig

    # 각 split에 대해 전처리 적용
    tokenized_dataset = dataset.map(
        lambda examples: preprocess_function(
            examples, tokenizer, config.MAX_SOURCE_LENGTH, config.MAX_TARGET_LENGTH
        ),
        batched=True,
        remove_columns=dataset["train"].column_names,  # 원본 컬럼 제거
    )

    return tokenized_dataset


# ============================================================================
# 5. 평가 메트릭 설정
# ============================================================================


def compute_metrics(eval_pred, tokenizer):
    # ROUGE 메트릭 로드
    rouge = evaluate.load("rouge")

    predictions, labels = eval_pred

    # 디코딩 (패딩 토큰 제거)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # labels에서 -100 (무시할 토큰)을 pad_token_id로 변환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE 점수 계산
    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    # 점수를 백분율로 변환
    result = {k: round(v * 100, 4) for k, v in result.items()}

    return result


# ============================================================================
# 6. 훈련 설정 및 실행
# ============================================================================


def setup_training_args(output_dir=None):
    config = KoBARTFineTuningConfig

    if output_dir is None:
        output_dir = config.OUTPUT_DIR

    training_args = Seq2SeqTrainingArguments(
        # 출력 설정
        output_dir=output_dir,
        logging_dir=config.LOGGING_DIR,
        # 훈련 설정
        num_train_epochs=config.NUM_EPOCHS,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        learning_rate=config.LEARNING_RATE,
        warmup_steps=config.WARMUP_STEPS,
        # 평가 및 저장 설정
        eval_strategy="steps",  # 최신 transformers에서는 eval_strategy 사용
        eval_steps=config.EVAL_STEPS,
        save_strategy="steps",  # 저장 전략도 명시적으로 설정
        save_steps=config.SAVE_STEPS,
        logging_steps=config.LOGGING_STEPS,
        # 최적화 설정
        gradient_accumulation_steps=2,
        fp16=torch.cuda.is_available(),  # GPU가 있으면 mixed precision 사용
        # MPS(Metal) 환경에서는 multiprocessing이 제대로 작동하지 않으므로 0으로 설정
        dataloader_num_workers=0 if torch.backends.mps.is_available() else 4,
        # MPS 환경에서는 pin_memory를 비활성화하여 경고 제거
        dataloader_pin_memory=False if torch.backends.mps.is_available() else True,
        # 모델 저장 설정
        save_total_limit=3,  # 최대 3개의 체크포인트만 유지
        load_best_model_at_end=True,
        metric_for_best_model="rouge1",  # ROUGE-1 점수를 기준으로 최적 모델 선택
        # 생성 설정
        predict_with_generate=True,  # 평가 시 생성된 텍스트로 메트릭 계산
        # 기타 설정
        report_to="tensorboard",  # TensorBoard 로깅
        remove_unused_columns=False,
    )

    return training_args


def train_kobart(
    dataset=None,
    model_name=None,
    output_dir=None,
    use_sample_data=True,
    json_file=None,
):
    if dataset is None:
        dataset = load_custom_dataset(json_file)

    print(f"  훈련 데이터: {len(dataset['train'])}개")
    print(f"  검증 데이터: {len(dataset['validation'])}개")
    print(f"  테스트 데이터: {len(dataset['test'])}개")

    # 2. 토크나이저 및 모델 로드
    tokenizer, model = load_tokenizer_and_model(model_name)

    # 3. 데이터 전처리
    tokenized_dataset = prepare_dataset(dataset, tokenizer)

    # 4. 데이터 콜레이터 설정
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer, model=model, padding=True
    )

    # 5. 훈련 인자 설정
    training_args = setup_training_args(output_dir)

    # 6. Trainer 생성
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,  # 최신 transformers에서는 processing_class 사용
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
    )

    # 7. 훈련 실행
    print("\n[7단계] 훈련 시작...")
    train_result = trainer.train()

    # 8. 최종 평가
    eval_results = trainer.evaluate(tokenized_dataset["test"])
    print(f"\n테스트 세트 평가 결과:")
    for key, value in eval_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")

    # 9. 모델 저장
    print(f"\n[9단계] 모델 저장 중: {training_args.output_dir}")
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

    print("\n파인튜닝 완료!")

    return trainer, tokenizer, model


# ============================================================================
# 7. 추론 함수
# ============================================================================


def generate_summary(
    model,
    tokenizer,
    text,
    max_length=None,
    num_beams=None,
):
    config = KoBARTFineTuningConfig

    if max_length is None:
        max_length = config.MAX_TARGET_LENGTH
    if num_beams is None:
        num_beams = config.NUM_BEAMS

    # 입력 토큰화
    inputs = tokenizer(
        text,
        max_length=config.MAX_SOURCE_LENGTH,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
        return_token_type_ids=False,  # BART는 token_type_ids를 사용하지 않음
    )

    # GPU로 이동
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # token_type_ids가 있다면 제거 (BART는 사용하지 않음)
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    # 요약 생성
    model.eval()
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=config.EARLY_STOPPING,
            no_repeat_ngram_size=2,  # 반복 방지
            repetition_penalty=1.2,  # 반복 페널티
        )

    # 디코딩
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return summary


def load_finetuned_model(model_path):
    print(f"파인튜닝된 모델 로드 중: {model_path}")
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
    model = BartForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model


# ============================================================================
# 8. 메인 실행 함수
# ============================================================================

if __name__ == "__main__":
    # 랜덤 시드 설정
    torch.manual_seed(42)
    np.random.seed(42)

    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"사용 장치: {device}")

    # JSON 파일 경로 설정
    json_file = "data/기형도-시집.json"

    # 파인튜닝 실행 (JSON 파일 사용)
    trainer, tokenizer, model = train_kobart(json_file=json_file, use_sample_data=False)

    # 추론 테스트
    print("\n" + "=" * 60)
    print("추론 테스트")
    print("=" * 60)

    # 시집에서 샘플 텍스트 사용
    test_text = (
        "아침 저녁으로 샛강에 자욱이 안개가 낀다. "
        "이 읍에 처음 와본 사람은 누구나 거대한 안개의 강을 거쳐야 한다. "
        "앞서간 일행들이 천천히 지워질 때까지 쓸쓸한 가축들처럼 그들은 "
        "그 긴 방죽 위에 서 있어야 한다. 문득 저 홀로 안개의 빈 구멍 속에 "
        "갇혀 있음을 느끼고 경악할 때까지."
    )

    print(f"\n원본 텍스트:\n{test_text}\n")

    summary = generate_summary(model, tokenizer, test_text)
    print(f"생성된 요약:\n{summary}\n")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


사용 장치: cpu
  훈련 데이터: 23개
  검증 데이터: 4개
  테스트 데이터: 6개
모델 로딩 중: gogamza/kobart-base-v2


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


토크나이저 어휘 크기: 30000
모델 파라미터 수: 123,859,968


Map:   0%|          | 0/23 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6 [00:00<?, ? examples/s]


[7단계] 훈련 시작...


Step,Training Loss,Validation Loss


c:\Users\user\miniconda3\envs\torch2\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
c:\Users\user\miniconda3\envs\torch2\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



테스트 세트 평가 결과:
  eval_loss: 22.8824
  eval_rouge1: 0.0000
  eval_rouge2: 0.0000
  eval_rougeL: 0.0000
  eval_rougeLsum: 0.0000
  eval_runtime: 31.4575
  eval_samples_per_second: 0.1910
  eval_steps_per_second: 0.0320
  epoch: 3.0000

[9단계] 모델 저장 중: ./kobart-finetuned

파인튜닝 완료!

추론 테스트

원본 텍스트:
아침 저녁으로 샛강에 자욱이 안개가 낀다. 이 읍에 처음 와본 사람은 누구나 거대한 안개의 강을 거쳐야 한다. 앞서간 일행들이 천천히 지워질 때까지 쓸쓸한 가축들처럼 그들은 그 긴 방죽 위에 서 있어야 한다. 문득 저 홀로 안개의 빈 구멍 속에 갇혀 있음을 느끼고 경악할 때까지.

생성된 요약:
일들들들이 천천히 지워질 때까지 쓸쓸한 가축들처럼 그들은 그 긴 방죽 위에 서 있어야 한다. 문득 저 홀로 안개의 빈 구멍 속에 갇혀 있음을 느끼고 경악할 때까지. 문득. 지나 지나 샛 강에 자욱이 안개가 낀다. 이 읍은 아침 저녁으로 달아   은 은 안위가 안무가 껴다. 자 선 선은 안이가 안트가  끼다.이 읍에 처음 와본 사람은 누구나 거대한 큰 큰 안욱 이 안대가  끼고다. 나 읍 에 처음 나 나 이 이 지난 지난 다 다 지나 남 남들 같은 이렇게 이렇게 그들은



In [12]:
%pip install -U nltk rouge_score absl-py evaluate

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached click-8.3.1-py3-none-any.whl (108 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=25027 sha256=abb533ab2776ac4b58363e55d1140b09489f514c7b91632819d180a0114b3391
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\44\af\da\5ffc433e2786f0b1a9c6f458d5fb8f611d8eb332387f

In [8]:
%pip install tensorboardX


   ---------------------------------------- 0/2 [protobuf]
   -------------------- ------------------- 1/2 [tensorboardX]
   ---------------------------------------- 2/2 [tensorboardX]

Note: you may need to restart the kernel to use updated packages.
